In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import collections

In [2]:
# read picture as GRAYSCALE
img=cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [3]:
# binary image(white=>0 ; black=>1)
def binarize(image):
    answer=np.zeros(image.shape,dtype='int')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j]>127:
                answer[i][j]=0
            else:
                answer[i][j]=1
    return answer

In [4]:
# invert binary image (1<->0)
def binary_image_invert(image):
    temp=np.zeros(image.shape,dtype='int')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j]==0:
                temp[i][j]=1
                
    return temp

In [5]:
# negative=>abs ,positive=>0
def operator(top_diff,buttom_diff,left_diff,right_diff,shape):
    return_list=[]
    
    if top_diff<0:
        return_list.append(abs(top_diff))
    else:
        return_list.append(0)
        
    if buttom_diff>shape[0]-1:
        return_list.append(buttom_diff-shape[0]+1)
    else:
        return_list.append(0)
    
    if left_diff<0:
        return_list.append(abs(left_diff))
    else:
        return_list.append(0)
        
    if right_diff>shape[1]-1:
        return_list.append(right_diff-shape[1]+1)
    else:
        return_list.append(0)
        
    return return_list
        

In [6]:
# dilation_binary(0 is the target)
def dilation_binary(bin_image,kernel,kernel_center):
    
    binary_image=np.zeros(bin_image.shape)
    binary_image[0:bin_image.shape[0],0:bin_image.shape[1]]=bin_image[0:bin_image.shape[0],0:bin_image.shape[1]]
    
    # get the 0s' position
    binary_TF=(binary_image==0)
    
    #unique, counts = np.unique(binary_image, return_counts=True)
    #print(dict(zip(unique, counts)))
    #cv2.imwrite('dilation_before.bmp',binary_image)

    return_image=np.zeros(binary_image.shape)
    
    top_border=-1*kernel_center[0]
    buttom_border=kernel.shape[0]-1-kernel_center[0]
    left_border=-1*kernel_center[1]
    right_border=kernel.shape[1]-1-kernel_center[1]
    
    #print("border",top_border,buttom_border,left_border,right_border)
    
    #print("start\n",binary_image)

    for i in range(binary_image.shape[0]):
        for j in range(binary_image.shape[1]):
            if binary_TF[i][j]:
         
                top_diff=i+top_border
                buttom_diff=i+buttom_border
                left_diff=j+left_border
                right_diff=j+right_border
                
                top_diff,buttom_diff,left_diff,right_diff=operator(top_diff,buttom_diff,left_diff,right_diff,binary_image.shape)
            
                binary_image[i+top_border+top_diff:i+buttom_border-buttom_diff+1,j+left_border+left_diff:j+right_border-right_diff+1]=binary_image[i+top_border+top_diff:i+buttom_border-buttom_diff+1,j+left_border+left_diff:j+right_border-right_diff+1]-kernel[top_diff:kernel.shape[0]-buttom_diff,left_diff:kernel.shape[1]-right_diff]
                
    
    for i in range(binary_image.shape[0]):
        for j in range(binary_image.shape[1]):
            if binary_image[i][j]>0:
                return_image[i][j]=1
                
    final=np.zeros(return_image.shape,dtype='int')
    for i in range(return_image.shape[0]):
        for j in range(return_image.shape[1]):
            if return_image[i][j]==0:
                final[i][j]=255
        
    #cv2.imwrite('dilation.bmp',final)
    
    #unique, counts = np.unique(return_image, return_counts=True)
    #print(dict(zip(unique, counts)))

            
    return final

In [7]:
# erosion_binary
def erosion_binary(bin_image,kernel,kernel_center):
    
    binary_image=np.zeros(bin_image.shape)
    binary_image[0:bin_image.shape[0],0:bin_image.shape[1]]=bin_image[0:bin_image.shape[0],0:bin_image.shape[1]]

    inverted_image=binary_image_invert(binary_image)
    # 0 if not erotion
    return_image=np.ones(binary_image.shape)
    
    #unique, counts = np.unique(binary_image, return_counts=True)
    #print(dict(zip(unique, counts)))
    
    
    top_border=-1*kernel_center[0]
    buttom_border=kernel.shape[0]-1-kernel_center[0]
    left_border=-1*kernel_center[1]
    right_border=kernel.shape[1]-1-kernel_center[1]
    
    
    for i in range(inverted_image.shape[0]):
        for j in range(inverted_image.shape[1]):
            top_diff=i+top_border
            buttom_diff=i+buttom_border
            left_diff=j+left_border
            right_diff=j+right_border
                
            top_diff,buttom_diff,left_diff,right_diff=operator(top_diff,buttom_diff,left_diff,right_diff,inverted_image.shape)
            if top_diff>0 or buttom_diff>0 or left_diff>0 or right_diff>0:
                continue
            
            check_matrix=inverted_image[i+top_border:i+buttom_border+1,j+left_border:j+right_border+1]>=kernel[0:kernel.shape[0],0:kernel.shape[1]]
                
            if not(False in check_matrix):
                return_image[i][j]=0
                
    final=np.zeros(return_image.shape,dtype='int')
    for i in range(return_image.shape[0]):
        for j in range(return_image.shape[1]):
            if return_image[i][j]==0:
                final[i][j]=255

    #cv2.imwrite('erosion.bmp',final)
    
    #unique, counts = np.unique(final, return_counts=True)
    #print(dict(zip(unique, counts)))
    
    return final
    

In [8]:
def hit_and_miss(bin_image):
    
    binary_image=np.zeros(bin_image.shape)
    binary_image[0:bin_image.shape[0],0:bin_image.shape[1]]=bin_image[0:bin_image.shape[0],0:bin_image.shape[1]]
    
    kernel=np.array([[1,1],[0,1]],dtype='int')
    kernel_J_center=[0,1]
    kernel_K_center=[1,0]
    
    binary_image_complement=binary_image_invert(binary_image)
    
    A_erosion_J=erosion_binary(binary_image,kernel,kernel_J_center)
    AComplement_erosion_K=erosion_binary(binary_image_complement,kernel,kernel_K_center)
    
    final_image=np.zeros(binary_image.shape,dtype='int')
    for i in range(binary_image.shape[0]):
        for j in range(binary_image.shape[1]):
            if A_erosion_J[i][j]==255 and AComplement_erosion_K[i][j]==255:
                final_image[i][j]=255
    
    return final_image

    

In [9]:
kernel=np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]])
#print(kernel)
try_image=np.array([[0,0,0,255,0],[255,0,255,255,255],[255,255,255,255,255],[0,255,0,255,0],[0,255,0,255,0]])
#print(try_image)
try_kernel=np.array([[1],[1]]).reshape(2,1)
#print(try_kernel)

binary_image=binarize(img)

print("dilationing...")
dilation_result=dilation_binary(binary_image,kernel,[2,2])
print("erosioning...")
erosion_result=erosion_binary(binary_image,kernel,[2,2])
print("opining...")
opining_result=dilation_binary(binarize(erosion_binary(binary_image,kernel,[2,2])),kernel,[2,2])
print("closing...")
closing_result=erosion_binary(binarize(dilation_binary(binary_image,kernel,[2,2])),kernel,[2,2])
print("hit_and_missing...")
hit_and_miss_result=hit_and_miss(binary_image)



dilationing...
erosioning...
opining...
closing...
hit_and_missing...


In [10]:
cv2.imwrite('dilation_new.bmp',dilation_result)
cv2.imwrite('erosion_new.bmp',erosion_result)
cv2.imwrite('opining_new.bmp',opining_result)
cv2.imwrite('closing_new.bmp',closing_result)
cv2.imwrite('hit_and_miss_new.bmp',hit_and_miss_result)

True